In [6]:
import xarray_beam as xbeam
import xarray
import apache_beam as beam
import pandas as pd
import numpy as np 

ds = xarray.open_zarr('gcs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2')
print(ds.nbytes / (10**9))

534392.622804012


In [7]:
import dask.array as da

AIR_VARIABLE = ['geopotential', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']

SURFACE_VARIABLE = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_cloud_cover', 'total_precipitation']

VARIABLES = AIR_VARIABLE + SURFACE_VARIABLE


start_date = pd.to_datetime('2016-12-31')
end_date = pd.to_datetime('2021-12-31')

lat_min, lat_max = (32, 39.3)
lon_min, lon_max = (124, 131.4)


# 해당 범위에 속하는 위도와 경도의 인덱스 찾기
lat_indices = np.where((ds.latitude >= lat_min) & (ds.latitude <= lat_max))[0]
lon_indices = np.where((ds.longitude >= lon_min) & (ds.longitude <= lon_max))[0]

ds = ds.sel(time=slice(start_date, end_date))

ds = ds.isel(latitude=lat_indices, longitude=lon_indices).sortby('latitude', ascending=True)


def standardize(data):
    mean = data.mean(dim=['time', 'latitude', 'longitude'], keep_attrs=True)
    std = data.std(dim=['time', 'latitude', 'longitude'], keep_attrs=True)
    standardized_data = (data - mean) / std
    return standardized_data, mean, std

for val in VARIABLES:
    standardized_data, mean, std = standardize(ds[val])
    ds[val] = standardized_data
    ds[val + "_mean"] = mean
    ds[val + "_std"] = std


ds_stacked = ds.stack(spatial=('latitude', 'longitude'))

variables_with_level = [var for var in ds_stacked.data_vars if 'level' in ds_stacked[var].dims]

level_vars = [var for var in ds_stacked.variables if 'level' in ds_stacked[var].dims and var in mean]

# 각 level 차원마다 새 변수 생성
for var_name in level_vars:
    for level in ds_stacked.level:
        # 새 변수 이름 형식: 원본변수명_level값
        new_var_name = f"{var_name}_level_{level.values}"
        
        # 선택한 level에 대한 데이터를 새 변수로 할당
        ds_stacked[new_var_name] = ds_stacked[var_name].sel(level=level)
        
        # 필요하다면, 새로운 변수에서 level 차원을 제거
        ds_stacked[new_var_name] = ds_stacked[new_var_name].drop_vars('level', errors='ignore')
    

ds_stacked = ds_stacked.drop_vars(level_vars)

print(ds_stacked.keys)



<bound method Mapping.keys of <xarray.Dataset>
Dimensions:                                           (time: 43825, spatial: 900)
Coordinates:
  * time                                              (time) datetime64[ns] 2...
  * spatial                                           (spatial) object MultiIndex
  * latitude                                          (spatial) float32 32.0 ...
  * longitude                                         (spatial) float32 124.0...
Data variables: (12/740)
    10m_u_component_of_wind                           (time, spatial) float32 dask.array<chunksize=(1, 900), meta=np.ndarray>
    10m_v_component_of_wind                           (time, spatial) float32 dask.array<chunksize=(1, 900), meta=np.ndarray>
    2m_temperature                                    (time, spatial) float32 dask.array<chunksize=(1, 900), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (spatial) float32 dask.array<chunksize=(900,), meta=np.ndarray>
    anisotro

In [9]:
print(ds_stacked['vertical_velocity_level_875'])

<xarray.DataArray 'vertical_velocity_level_875' (time: 43825, spatial: 900)>
dask.array<getitem, shape=(43825, 900), dtype=float32, chunksize=(1, 900), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2016-12-31 ... 2021-12-31
  * spatial    (spatial) object MultiIndex
  * latitude   (spatial) float32 32.0 32.0 32.0 32.0 ... 39.25 39.25 39.25 39.25
  * longitude  (spatial) float32 124.0 124.2 124.5 124.8 ... 130.8 131.0 131.2


In [ ]:
ds_stacked = ds_stacked.drop_vars(AIR_VARIABLE)

print(len(list(ds_stacked.keys())))

In [ ]:
print(ds_stacked['vertical_velocity_level_875'])

In [9]:
print(ds_stacked)

dataset = []
for val in ds_stacked.keys():
    dataset.append(ds_stacked[val].values)

<xarray.Dataset>
Dimensions:                       (time: 43825, spatial: 900, level: 37)
Coordinates:
  * level                         (level) int64 1 2 3 5 7 ... 925 950 975 1000
  * time                          (time) datetime64[ns] 2016-12-31 ... 2021-1...
  * spatial                       (spatial) object MultiIndex
  * latitude                      (spatial) float32 32.0 32.0 ... 39.25 39.25
  * longitude                     (spatial) float32 124.0 124.2 ... 131.0 131.2
Data variables: (12/39)
    10m_u_component_of_wind       (time, spatial) float32 dask.array<chunksize=(64, 900), meta=np.ndarray>
    10m_v_component_of_wind       (time, spatial) float32 dask.array<chunksize=(64, 900), meta=np.ndarray>
    2m_temperature                (time, spatial) float32 dask.array<chunksize=(64, 900), meta=np.ndarray>
    geopotential                  (time, level, spatial) float32 dask.array<chunksize=(64, 37, 900), meta=np.ndarray>
    land_sea_mask                 (spatial) float32 da

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 2차원 넘파이 배열 생성

# 히트맵 생성
plt.imshow(sea_data, cmap='hot', interpolation='nearest')
plt.colorbar()  # 색상 바 추가
plt.show()

In [ ]:
HAS_LEVEL_VARIABLE = [ 'geopotential', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']

# 새 차원을 추가하고 데이터 변수를 결합
data_arrays = [ds[var].expand_dims('variable').assign_coords(variable=[var]) for var in HAS_LEVEL_VARIABLE]
combined_ds = xarray.concat(data_arrays, dim='variable')
# 결과 확인
stacked_ds = combined_ds.stack(variable_level=('variable', 'level'))
stacked_ds = stacked_ds.stack(hidden_dim=('latitude', 'longitude'))
print(stacked_ds.values.shape)

In [ ]:
HAS_LEVEL_VARIABLE = [ 'geopotential', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']

# 새 차원을 추가하고 데이터 변수를 결합
data_arrays = [ds[var].expand_dims('variable').assign_coords(variable=[var]) for var in HAS_LEVEL_VARIABLE]
combined_ds = xarray.concat(data_arrays, dim='variable')
# 결과 확인
stacked_ds = combined_ds.stack(variable_level=('variable', 'level'))

In [ ]:
print(stacked_ds.values)

In [ ]:
new_dataset = ds.isel(latitude=lat_indices, longitude=lon_indices)

In [ ]:
print(new_dataset)

In [ ]:
variable = ['geopotential', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']
arr = ds[variable]

In [ ]:
print(arr)

In [ ]:
# 새 차원을 추가하고 데이터 변수를 결합
data_arrays = [arr[var].expand_dims('variable').assign_coords(variable=[var]) for var in variable]
combined_ds = xarray.concat(data_arrays, dim='variable')
# 결과 확인
stacked_ds = combined_ds.stack(variable_level=('variable', 'level'))
stacked_ds = stacked_ds.stack(hidden_dim=('latitude', 'longitude'))


In [ ]:
print(stacked_ds)

In [ ]:
arr = stacked_ds.compute()
print(arr)

In [ ]:
import numpy as np

lat_min, lat_max = 32.2, 39.0
lon_min, lon_max = 124.2, 131

# 해당 범위에 속하는 위도와 경도의 인덱스 찾기
lat_indices = np.where((ds.latitude >= lat_min) & (ds.latitude <= lat_max))[0]
lon_indices = np.where((ds.longitude >= lon_min) & (ds.longitude <= lon_max))[0]

print("Latitude indices:", len(lat_indices))
print("Longitude indices:", len(lon_indices))

In [ ]:
ds_kor = arr.isel(latitude=lat_indices, longitude=lon_indices)
print(ds_kor)

In [ ]:
dar = ds_kor['2m_temperature']
dar = dar.sel(level=100)

data = dar.to_numpy()
print(data.shape)

In [ ]:
print(data)

In [ ]:
import pandas as pd
dar = ds['2m_temperature']
i2 = pd.to_datetime(str(2018) + '-03-01T00:00:00.000000000') 
dar = dar.sel(time=i2)
print(dar)

In [ ]:
ds_kor = dar.isel(latitude=lat_indices, longitude=lon_indices)
print(ds_kor.latitude)
print(ds_kor.longitude)


In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (6,6) #차트 사이즈
from mpl_toolkits.basemap import Basemap, cm
m = Basemap(llcrnrlon=min(ds_kor.longitude),llcrnrlat=min(ds_kor.latitude),urcrnrlon=max(ds_kor.longitude),urcrnrlat=max(ds_kor.latitude), resolution='i',projection='cyl',lon_0=(max(ds_kor.longitude)-min(ds_kor.longitude)) / 2,lat_0=(max(ds_kor.latitude)-min(ds_kor.latitude)) / 2)
ds_kor.plot()
m.drawcoastlines()